In [1]:
import sys
import os
from typing import Tuple
import pandas

sys.path.append('/home/danillorp/Área de Trabalho/github/fema/src/')


import fema_semi
import fema_classifier
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import preprocessing


/tmp/ipykernel_81024/2461910642.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [12]:
ml_datasets = ['arrhythmia','Titanic','bank-marketing','vehicle','adult','airlines','PhishingWebsites',
               'kc1','pc1','cmc','mfeat-factors','KDDCup09_appetency']

#'credit-g','SpeedDating','blood-transfusion-service-center','mnist_784', 'satellite','one-hundred-plants-texture','steel-plates-fault',
#'kr-vs-kp',
print('The experiment will run for ',len(ml_datasets))        

The experiment will run for  12


In [13]:
def experiment(dataset_name:str, data_x:pandas.core.frame.DataFrame, data_y:pandas.core.series.Series, test_size:float):
    train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=test_size)
    uknw_x, test_x, uknw_y, test_y = train_test_split(test_x, test_y, test_size=(1-test_size))

    print(dataset_name)
    print('train_size',len(train_x),'classes:',set(train_y))
    print('test_size',len(test_x),'classes:',set(test_y))
    print('uknw_size',len(uknw_x),'classes:',set(uknw_y))

    #train_y = train_y.to_numpy()
    #test_y = test_y.to_numpy()
    #uknw_y = uknw_y.to_numpy()

    train_y = train_y[:,np.newaxis]
    test_y = test_y[:,np.newaxis]
    uknw_y = uknw_y[:,np.newaxis]

    
    scaler = StandardScaler()

    train_x = scaler.fit_transform(train_x)
    test_x = scaler.transform(test_x)
    uknw_x = scaler.transform(uknw_x)

    #print(type(train_y),train_y.shape,train_y[:,0])

    model_fema_original = fema_classifier.FEMaClassifier(k=10,basis=fema_classifier.Basis.shepardBasis)
    model_fema_original.fit(train_x,train_y)

    pred, confidence_level = model_fema_original.predict(test_x,2)

    cm_fema_original = confusion_matrix(test_y,pred)
    bal_acc_original = balanced_accuracy_score(test_y, pred)
    acc_original = accuracy_score(test_y, pred)

    print(dataset_name,cm_fema_original, bal_acc_original, acc_original)

    model_semi = fema_semi.FEMaSemiSupervisedClassifier(k=10,basis=fema_semi.Basis.shepardBasis)
    model_semi.fit(train_x,train_y,uknw_x,2)
    
    pred, confidence_level = model_semi.predict(test_x,2)
    
    cm_fema_semi = confusion_matrix(test_y,pred)
    bal_acc_semi = balanced_accuracy_score(test_y, pred)
    acc_semi = accuracy_score(test_y, pred)
    
    
    print(cm_fema_semi, bal_acc_semi, acc_semi)




In [ ]:
for dataset in ml_datasets: 
    data = fetch_openml(dataset,version=1)
    X, y = data['data'], data['target']

    X = X.select_dtypes(include=np.number)

    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    
    print(dataset, X.shape, y.shape, type(X), type(y))
    print(X.dtypes)
    experiment(dataset_name=dataset, data_x=X, data_y=y, test_size=0.20)

arrhythmia (452, 206) (452,) <class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
age                 int64
height              int64
weight              int64
QRSduration         int64
PRinterval          int64
                   ...   
chV6_SPwaveAmp    float64
chV6_PwaveAmp     float64
chV6_TwaveAmp     float64
chV6_QRSA         float64
chV6_QRSTA        float64
Length: 206, dtype: object
arrhythmia
train_size 361 classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
test_size 73 classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12}
uknw_size 18 classes: {0, 3, 4, 5, 7}
arrhythmia [[32  0  0  0  0  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  0  0  0]
 [ 7  0  0  0  0  1  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  